In [1]:
# Imports
import berserk
import pandas as pd
import datetime
import time
from tqdm import tqdm
import os
from typing import Dict, Any, Optional

In [2]:
API_TOKEN = os.getenv('API_TOKEN')

session = berserk.TokenSession(API_TOKEN)
client = berserk.Client(session=session)

In [ ]:
start_users_set = set()
blitz_users_list = []
bullet_users_list = []
classical_users_list = []
rapid_users_list = []
max_amount_users = 200
cnt_games_by_user = 200
time_for_sleep = 0.025

In [3]:
try:
    blitz_users_list = [i.get('id') for i in client.users.get_leaderboard(
        count=max_amount_users, perf_type='blitz')]
    start_users_set = start_users_set.union(blitz_users_list)
except Exception as e:
    print(f"Ошибка при получении списка пользователей для 'blitz': {e}")

try:
    bullet_users_list = [i.get('id') for i in client.users.get_leaderboard(
        count=max_amount_users, perf_type='bullet')]
    start_users_set = start_users_set.union(bullet_users_list)
except Exception as e:
    print(f"Ошибка при получении списка пользователей для 'bullet': {e}")

try:
    classical_users_list = [i.get('id') for i in client.users.get_leaderboard(
        count=max_amount_users, perf_type='classical')]
    start_users_set = start_users_set.union(classical_users_list)
except Exception as e:
    print(f"Ошибка при получении списка пользователей для 'classical': {e}")

try:
    rapid_users_list = [i.get('id') for i in client.users.get_leaderboard(
        count=max_amount_users, perf_type='rapid')]
    start_users_set = start_users_set.union(rapid_users_list)
except Exception as e:
    print(f"Ошибка при получении списка пользователей для 'rapid': {e}")

In [4]:
def transform_game_by_user(game_data: Dict[str, Any]) -> Dict[str, Optional[Any]]:
    """
    Преобразует данные шахматной партии в удобный формат.
    """
    return {
        'Game_ID': game_data.get('id'),
        'Rated': game_data.get('rated', False),
        'Variant': game_data.get('variant'),
        'Speed': game_data.get('speed'),
        'Perf': game_data.get('perf'),
        'Created_At': game_data.get('createdAt').isoformat() if isinstance(game_data.get('createdAt'), datetime.datetime) else None,
        'Last_Move_At': game_data.get('lastMoveAt').isoformat() if isinstance(game_data.get('lastMoveAt'), datetime.datetime) else None,
        'Game_Status': game_data.get('status'),
        'Source': game_data.get('source'),
        'Winner': game_data.get('winner'),
        'Opening_Name': game_data.get('opening', {}).get('name'),
        'Opening_ECO': game_data.get('opening', {}).get('eco'),
        'Opening_PLY': game_data.get('opening', {}).get('ply'),
        'White_Player': game_data.get('players', {}).get('white', {}).get('user', {}).get('id'),
        'White_Title': game_data.get('players', {}).get('white', {}).get('user', {}).get('title'),
        'White_Rating': game_data.get('players', {}).get('white', {}).get('rating', None),
        'White_Inaccuracy': game_data.get('players').get('white', {}).get('analysis').get('inaccuracy'),
        'White_Mistake': game_data.get('players').get('white', {}).get('analysis').get('mistake'),
        'White_Blunder': game_data.get('players').get('white', {}).get('analysis').get('blunder'),
        'White_ACPL': game_data.get('players').get('white', {}).get('analysis').get('acpl'),
        'Black_Player': game_data.get('players', {}).get('black', {}).get('user', {}).get('id'),
        'Black_Title': game_data.get('players', {}).get('black', {}).get('user', {}).get('title'),
        'Black_Rating': game_data.get('players', {}).get('black', {}).get('rating', None),
        'Black_Inaccuracy': game_data.get('players').get('black', {}).get('analysis').get('inaccuracy'),
        'Black_Mistake': game_data.get('players').get('black', {}).get('analysis').get('mistake'),
        'Black_Blunder': game_data.get('players').get('black', {}).get('analysis').get('blunder'),
        'Black_ACPL': game_data.get('players').get('black', {}).get('analysis').get('acpl'),
        'Clocks': game_data.get('clocks', []),
        'Analysis': game_data.get('analysis', []),
        'Total_Time_Init': game_data.get('clock', {}).get('initial', None),
        'Total_Time_Increment': game_data.get('clock', {}).get('increment', None),
        'Total_Time': game_data.get('clock', {}).get('totalTime', None),
        "Moves": game_data.get('moves', '')
    }

In [ ]:
blitz_df = pd.DataFrame()
processed_users_list = []


for user in tqdm(list(set(blitz_users_list)), desc="Обработка пользователей", unit="пользователь"):
    try:
        for game in client.games.export_by_player(
            max=cnt_games_by_user, username=user,
            analysed=True, perf_type=['blitz'],
            evals=True, opening=True, clocks=True, literate=True,
            pgn_in_json=True, tags=True, finished=True
        ):
            try:
                transformed_game = transform_game_by_user(game)
                time.sleep(time_for_sleep)
                if transformed_game:
                    df = pd.DataFrame([transformed_game])
                    blitz_df = pd.concat([blitz_df, df], ignore_index=True)
            except Exception as e:
                print(
                    f"Ошибка при преобразовании игры для пользователя {user}: {e}")

        processed_users_list.append(user)
    except Exception as e:
        print(f"Ошибка при получении игр для пользователя {user}: {e}")

Обработка пользователей: 100%|██████████| 5/5 [00:04<00:00,  1.16пользователь/s]


In [6]:
ultra_bullet_df = pd.DataFrame()

for user in tqdm(list(set(bullet_users_list)), desc="Обработка пользователей", unit="пользователь"):
    try:
        for game in client.games.export_by_player(
            max=cnt_games_by_user, username=user,
            analysed=True, perf_type=['ultraBullet'],
            evals=True, opening=True, clocks=True, literate=True,
            pgn_in_json=True, tags=True, finished=True
        ):
            try:
                transformed_game = transform_game_by_user(game)
                time.sleep(time_for_sleep)

                if transformed_game:
                    df = pd.DataFrame([transformed_game])
                    ultra_bullet_df = pd.concat(
                        [ultra_bullet_df, df], ignore_index=True)
            except Exception as e:
                print(
                    f"Ошибка при преобразовании игры для пользователя {user}: {e}")

        processed_users_list.append(user)
    except Exception as e:
        print(f"Ошибка при получении игр для пользователя {user}: {e}")

Обработка пользователей: 100%|██████████| 5/5 [00:02<00:00,  1.75пользователь/s]


In [7]:
bullet_df = pd.DataFrame()

for user in tqdm(list(set(bullet_users_list)), desc="Обработка пользователей", unit="пользователь"):
    try:
        for game in client.games.export_by_player(
            max=cnt_games_by_user, username=user,
            analysed=True, perf_type=['bullet'],
            evals=True, opening=True, clocks=True, literate=True,
            pgn_in_json=True, tags=True, finished=True
        ):
            try:
                transformed_game = transform_game_by_user(game)
                time.sleep(time_for_sleep)

                if transformed_game:
                    df = pd.DataFrame([transformed_game])
                    bullet_df = pd.concat(
                        [bullet_df, df], ignore_index=True)
            except Exception as e:
                print(
                    f"Ошибка при преобразовании игры для пользователя {user}: {e}")

        processed_users_list.append(user)
    except Exception as e:
        print(f"Ошибка при получении игр для пользователя {user}: {e}")

Обработка пользователей:  60%|██████    | 3/5 [00:00<00:00,  4.71пользователь/s]

Ошибка при получении игр для пользователя mishka_the_great: 429 Client Error: Too Many Requests for url: https://lichess.org/api/games/user/mishka_the_great?max=5&perfType=bullet&analysed=True&pgnInJson=True&tags=True&clocks=True&evals=True&opening=True&finished=True&literate=True
Ошибка при получении игр для пользователя ediz_gurel: 429 Client Error: Too Many Requests for url: https://lichess.org/api/games/user/ediz_gurel?max=5&perfType=bullet&analysed=True&pgnInJson=True&tags=True&clocks=True&evals=True&opening=True&finished=True&literate=True
Ошибка при получении игр для пользователя azzaro25: 429 Client Error: Too Many Requests for url: https://lichess.org/api/games/user/azzaro25?max=5&perfType=bullet&analysed=True&pgnInJson=True&tags=True&clocks=True&evals=True&opening=True&finished=True&literate=True


Обработка пользователей: 100%|██████████| 5/5 [00:01<00:00,  4.95пользователь/s]

Ошибка при получении игр для пользователя chess-art-us: 429 Client Error: Too Many Requests for url: https://lichess.org/api/games/user/chess-art-us?max=5&perfType=bullet&analysed=True&pgnInJson=True&tags=True&clocks=True&evals=True&opening=True&finished=True&literate=True


In [8]:
rapid_df = pd.DataFrame()

for user in tqdm(list(set(rapid_users_list)), desc="Обработка пользователей", unit="пользователь"):
    try:
        for game in client.games.export_by_player(
            max=cnt_games_by_user, username=user,
            analysed=True, perf_type=['rapid'],
            evals=True, opening=True, clocks=True, literate=True,
            pgn_in_json=True, tags=True, finished=True
        ):
            try:
                transformed_game = transform_game_by_user(game)
                time.sleep(time_for_sleep)
                if transformed_game:
                    df = pd.DataFrame([transformed_game])
                    rapid_df = pd.concat(
                        [rapid_df, df], ignore_index=True)
            except Exception as e:
                print(
                    f"Ошибка при преобразовании игры для пользователя {user}: {e}")

        processed_users_list.append(user)
    except Exception as e:
        print(f"Ошибка при получении игр для пользователя {user}: {e}")

Обработка пользователей:  20%|██        | 1/5 [00:00<00:00,  9.56пользователь/s]

Ошибка при получении игр для пользователя just_no_fun: 429 Client Error: Too Many Requests for url: https://lichess.org/api/games/user/just_no_fun?max=5&perfType=rapid&analysed=True&pgnInJson=True&tags=True&clocks=True&evals=True&opening=True&finished=True&literate=True


Обработка пользователей:  60%|██████    | 3/5 [00:00<00:00,  9.47пользователь/s]

Ошибка при получении игр для пользователя ilqar_7474: 429 Client Error: Too Many Requests for url: https://lichess.org/api/games/user/ilqar_7474?max=5&perfType=rapid&analysed=True&pgnInJson=True&tags=True&clocks=True&evals=True&opening=True&finished=True&literate=True
Ошибка при получении игр для пользователя pultis12: 429 Client Error: Too Many Requests for url: https://lichess.org/api/games/user/pultis12?max=5&perfType=rapid&analysed=True&pgnInJson=True&tags=True&clocks=True&evals=True&opening=True&finished=True&literate=True


Обработка пользователей: 100%|██████████| 5/5 [00:00<00:00,  9.51пользователь/s]

Ошибка при получении игр для пользователя americanchampion: 429 Client Error: Too Many Requests for url: https://lichess.org/api/games/user/americanchampion?max=5&perfType=rapid&analysed=True&pgnInJson=True&tags=True&clocks=True&evals=True&opening=True&finished=True&literate=True
Ошибка при получении игр для пользователя tuzakli_egitim: 429 Client Error: Too Many Requests for url: https://lichess.org/api/games/user/tuzakli_egitim?max=5&perfType=rapid&analysed=True&pgnInJson=True&tags=True&clocks=True&evals=True&opening=True&finished=True&literate=True


In [9]:
classical_df = pd.DataFrame()

for user in tqdm(list(set(classical_users_list)), desc="Обработка пользователей", unit="пользователь"):
    try:
        for game in client.games.export_by_player(
            max=cnt_games_by_user, username=user,
            analysed=True, perf_type=['classical'],
            evals=True, opening=True, clocks=True, literate=True,
            pgn_in_json=True, tags=True, finished=True
        ):
            try:
                transformed_game = transform_game_by_user(game)
                time.sleep(time_for_sleep)
                if transformed_game:
                    df = pd.DataFrame([transformed_game])
                    classical_df = pd.concat(
                        [classical_df, df], ignore_index=True)
            except Exception as e:
                print(
                    f"Ошибка при преобразовании игры для пользователя {user}: {e}")

        processed_users_list.append(user)
    except Exception as e:
        print(f"Ошибка при получении игр для пользователя {user}: {e}")

Обработка пользователей:  60%|██████    | 3/5 [00:00<00:00,  4.78пользователь/s]

Ошибка при получении игр для пользователя vlad_lazarev79: 429 Client Error: Too Many Requests for url: https://lichess.org/api/games/user/vlad_lazarev79?max=5&perfType=classical&analysed=True&pgnInJson=True&tags=True&clocks=True&evals=True&opening=True&finished=True&literate=True
Ошибка при получении игр для пользователя tinea: 429 Client Error: Too Many Requests for url: https://lichess.org/api/games/user/tinea?max=5&perfType=classical&analysed=True&pgnInJson=True&tags=True&clocks=True&evals=True&opening=True&finished=True&literate=True


Обработка пользователей:  80%|████████  | 4/5 [00:22<00:08,  8.67s/пользователь]

Ошибка при получении игр для пользователя chesstheory64: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Обработка пользователей: 100%|██████████| 5/5 [00:43<00:00,  8.70s/пользователь]

Ошибка при получении игр для пользователя truemasterme: HTTPSConnectionPool(host='lichess.org', port=443): Max retries exceeded with url: /api/games/user/truemasterme?max=5&perfType=classical&analysed=True&pgnInJson=True&tags=True&clocks=True&evals=True&opening=True&finished=True&literate=True (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000024765CB1F10>, 'Connection to lichess.org timed out. (connect timeout=None)'))


In [10]:
common_df = pd.concat(
    [classical_df, blitz_df, rapid_df, bullet_df, ultra_bullet_df], ignore_index=True)
common_df['Speed'].value_counts()

Speed
blitz          25
ultraBullet    25
classical       5
bullet          5
Name: count, dtype: int64

In [12]:
common_df.filter(
    regex=r'^(?!.*Perf).*$').to_csv('final_games.csv', index=False)